### Install and import require packages

In [127]:
import os, time
%matplotlib inline
import matplotlib.pyplot as plt
import itertools
import pickle
import imageio

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from torchvision import datasets, transforms

### Set number of epochs, learning rate and batch size

In [128]:
batch_size = 128
lr = 0.0002
num_of_epochs = 15 # 20

### Load Dataset

In [129]:
img_size = 64
transform = transforms.Compose([
        transforms.Scale(img_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True, transform=transform),
    batch_size=batch_size, shuffle=True)

### Model

In [130]:
class ImageDiscriminator(nn.Module):
    def __init__(self, noise_vector_size=128):
        super(ImageDiscriminator, self).__init__()
        ##TODO: implement initial 4 Convolution layers using noise_vector_size with linear kernel(=4), stride(=2) and padding(=1)##
        ##TODO: add BatchNorm layers from second conv layer till fourth conv layer##
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=noise_vector_size, kernel_size=4, stride=2, padding=1)
        self.conv2 = nn.Conv2d(in_channels=noise_vector_size, out_channels=noise_vector_size*2, kernel_size=4, stride=2, padding=1)
        self.conv2_batchnorm = nn.BatchNorm2d(noise_vector_size*2)
        self.conv3 = nn.Conv2d(in_channels=noise_vector_size*2, out_channels=noise_vector_size*4, kernel_size=4, stride=2, padding=1)
        self.conv3_batchnorm = nn.BatchNorm2d(noise_vector_size*4)
        self.conv4 = nn.Conv2d(in_channels=noise_vector_size*4, out_channels=noise_vector_size*8, kernel_size=4, stride=2, padding=1)
        self.conv4_batchnorm = nn.BatchNorm2d(noise_vector_size*8)
        self.conv5 = nn.Conv2d(noise_vector_size * 8, 1, 4, 1, 0)

    def weight_init(self, mean, std):
        for m in self._modules:
            normal_init(self._modules[m], mean, std)

    def forward(self, input):
        ##TODO: implement forward pass (be cautious when adding BatchNorm layers)##
        x = self.conv1(input)
        x = F.relu(x)
        x = self.conv2(x)
        x = self.conv2_batchnorm(x)
        x = F.relu(x)
        x = self.conv3(x)
        x = self.conv3_batchnorm(x)
        x = F.relu(x)
        x = self.conv4(x)
        x = self.conv4_batchnorm(x)
        x = F.relu(x)
        x = F.sigmoid(self.conv5(x))

        return x

In [131]:
class ImageGenerator(nn.Module):
    def __init__(self, noise_vector_size=128):
        super(ImageGenerator, self).__init__()
        ##TODO: implement rest 4 Tranpose Convolution layers using noise_vector_size with linear kernel(=4), stride(=2) and padding(=1)##
        ##TODO: add BatchNorm layers from first conv layer till fourth conv layer##
        self.deconv1 = nn.ConvTranspose2d(100, noise_vector_size * 8, 4, 1, 0)
        self.deconv1_batchnorm = nn.BatchNorm2d(noise_vector_size*8)
        self.deconv2 = nn.ConvTranspose2d(in_channels=noise_vector_size * 8, out_channels=noise_vector_size * 4, kernel_size=4, stride=2, padding=1)
        self.deconv2_batchnorm = nn.BatchNorm2d(noise_vector_size*4)
        self.deconv3 = nn.ConvTranspose2d(in_channels=noise_vector_size * 4, out_channels=noise_vector_size * 2, kernel_size=4, stride=2, padding=1)
        self.deconv3_batchnorm = nn.BatchNorm2d(noise_vector_size*2)
        self.deconv4 = nn.ConvTranspose2d(in_channels=noise_vector_size * 2, out_channels=noise_vector_size, kernel_size=4, stride=2, padding=1)
        self.deconv4_batchnorm = nn.BatchNorm2d(noise_vector_size)
        self.deconv5 = nn.ConvTranspose2d(in_channels=noise_vector_size, out_channels=1, kernel_size=4, stride=2, padding=1)
        
        

    # weight_init
    def weight_init(self, mean, std):
        for m in self._modules:
            normal_init(self._modules[m], mean, std)

    # forward method
    def forward(self, input):
        ##TODO: implement forward pass (be cautious when adding BatchNorm layers)##
        
        x = self.deconv1(input)
        x = self.deconv1_batchnorm(x)
        x = F.relu(x)
        x = self.deconv2(x)
        x = self.deconv2_batchnorm(x)
        x = F.relu(x)
        x = self.deconv3(x)
        x = self.deconv3_batchnorm(x)
        x = F.relu(x)
        x = self.deconv4(x)
        x = self.deconv4_batchnorm(x)
        x = F.relu(x)
        x = self.deconv5(x)
        x = F.tanh(x)

        return x

In [132]:
###TODO: Replace normal weight initialization with xavier intialization##
def normal_init(m, mean, std):
    '''Init layer parameters.'''
    if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Conv2d):
        m.weight = torch.nn.init.xavier_normal(m.weight)
        if m.bias is not None:
            m.bias = torch.nn.init.constant(m.bias, 0)


    
def normal_init_old(m, mean, std):
    if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Conv2d):
        m.weight.data.normal_(mean, std)
        m.bias.data.zero_()

In [133]:
fixed_noise = torch.randn((5 * 5, 100)).view(-1, 100, 1, 1)
fixed_noise = Variable(fixed_noise.cuda(), volatile=True)

c:\program files\python35\lib\site-packages\ipykernel_launcher.py:2: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


In [134]:
def show_epoch_result(num_epoch, show = False, save = False, path = 'result.png', isFix=False):
    random_noise = torch.randn((5*5, 100)).view(-1, 100, 1, 1)
    random_noise = Variable(random_noise.cuda(), volatile=True)

    G.eval()
    if isFix:
        test_images = G(fixed_noise)
    else:
        test_images = G(random_noise)
    G.train()

    size_figure_grid = 5
    fig, ax = plt.subplots(size_figure_grid, size_figure_grid, figsize=(5, 5))
    for i, j in itertools.product(range(size_figure_grid), range(size_figure_grid)):
        ax[i, j].get_xaxis().set_visible(False)
        ax[i, j].get_yaxis().set_visible(False)

    for k in range(5*5):
        i = k // 5
        j = k % 5
        ax[i, j].cla()
        ax[i, j].imshow(test_images[k, 0].cpu().data.numpy(), cmap='gray')

    label = 'Epoch {0}'.format(num_epoch)
    fig.text(0.5, 0.04, label, ha='center')
    plt.savefig(path)

    if show:
        plt.show()
    else:
        plt.close()


def show_train_hist(hist, show = False, save = False, path = 'Train_hist.png'):
    ##TODO: implement show_train_hist function to plot losses histograms ##
    x = range(len(hist['D_losses']))

    y_discriminator = hist['D_losses']
    y_generator = hist['G_losses']

    plt.plot(x, y_discriminator, label='D_loss')
    plt.plot(x, y_generator, label='G_loss')

    plt.xlabel('Iterations')
    plt.ylabel('Loss values')

    plt.legend(loc=4)
    plt.grid(True)
    plt.tight_layout()
    
    
    if save:
        plt.savefig(path)

    if show:
        plt.show()
    else:
        plt.close()

In [135]:
G = ImageGenerator(128)
D = ImageDiscriminator(128)
G.weight_init(mean=0.0, std=0.02)
D.weight_init(mean=0.0, std=0.02)
G.cuda()
D.cuda()

c:\program files\python35\lib\site-packages\ipykernel_launcher.py:5: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  """
c:\program files\python35\lib\site-packages\ipykernel_launcher.py:7: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  import sys


ImageDiscriminator(
  (conv1): Conv2d(1, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv2_batchnorm): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv3_batchnorm): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(512, 1024, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv4_batchnorm): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(1024, 1, kernel_size=(4, 4), stride=(1, 1))
)

### Optimization

In [136]:
BCE_loss = nn.BCELoss()
G_optimizer = optim.Adam(G.parameters(), lr=lr, betas=(0.5, 0.999))
D_optimizer = optim.Adam(D.parameters(), lr=lr, betas=(0.5, 0.999))


### Results folder to save visualizations

In [137]:
if not os.path.isdir('results'):
    os.mkdir('results')
if not os.path.isdir('results/dcgan'):
    os.mkdir('results/dcgan')
if not os.path.isdir('results/dcgan/Random_results'):
    os.mkdir('results/dcgan/Random_results')
if not os.path.isdir('results/dcgan/Fixed_results'):
    os.mkdir('results/dcgan/Fixed_results')

train_hist = {}
train_hist['D_losses'] = []
train_hist['G_losses'] = []
train_hist['per_epoch_times'] = []
train_hist['total_time'] = []
num_iter = 0

### Training

In [138]:
print('Started training...')
start_time = time.time()
for epoch in range(num_of_epochs):
    D_losses = []
    G_losses = []
    epoch_start_time = time.time()
    for x_, _ in list(train_loader):
        print("Iteration", num_iter, "of", len(train_loader))
        
        ##TODO: fill-in training discriminator procedure (hint: similar to mlp_gan.ipynb implementation) ##
        
        D.zero_grad()
                
        batch = x_.size()[0]

        pred_real = torch.ones(batch)
        pred_fake = torch.zeros(batch)

        x_, pred_real, pred_fake = Variable(x_.cuda()), Variable(pred_real.cuda()), Variable(pred_fake.cuda())
        result_Discriminator = D(x_).squeeze()
        real_loss_Discriminator = BCE_loss(result_Discriminator, pred_real)

        z = torch.randn((batch, 100)).view(-1, 100, 1, 1)
        z = Variable(z.cuda())
        result_Generator = G(z)

        result_Discriminator = D(result_Generator).squeeze()
        fake_loss_Discriminator = BCE_loss(result_Discriminator, pred_fake)
        fake_score_Discriminator = result_Discriminator.data.mean()

        train_loss_Discriminator = real_loss_Discriminator + fake_loss_Discriminator

        train_loss_Discriminator.backward()
        D_optimizer.step()

        D_losses.append(train_loss_Discriminator.item())

        ##TODO: fill-in training generator procedure (hint: similar to mlp_gan.ipynb implementation) ##
        G.zero_grad()

        z = torch.randn((batch, 100)).view(-1, 100, 1, 1)
        z = Variable(z.cuda())

        result_Generator = G(z)
        result_Discriminator = D(result_Generator).squeeze()
        train_loss_Generator = BCE_loss(result_Discriminator, pred_real)
        train_loss_Generator.backward()
        G_optimizer.step()
        G_losses.append(train_loss_Generator.item())

        num_iter += 1
        

    epoch_end_time = time.time()
    per_epoch_time = epoch_end_time - epoch_start_time


    print('[%d/%d] - time: %.2f, loss_d: %.3f, loss_g: %.3f' % ((epoch + 1), num_of_epochs, per_epoch_time, torch.mean(torch.FloatTensor(D_losses)),
                                                              torch.mean(torch.FloatTensor(G_losses))))
    p = 'results/dcgan/Random_results/' + str(epoch + 1) + '.png'
    fixed_p = 'results/dcgan/Fixed_results/' + str(epoch + 1) + '.png'
    
    ##TODO: save the epoch results with fixed_noise and randon_noise ##
    show_epoch_result((epoch+1), save=True, path=p, isFix=False)
    show_epoch_result((epoch+1), save=True, path=fixed_p, isFix=True)
    train_hist['D_losses'].append(torch.mean(torch.FloatTensor(D_losses)))
    train_hist['G_losses'].append(torch.mean(torch.FloatTensor(G_losses)))
    train_hist['per_epoch_times'].append(per_epoch_time)

end_time = time.time()
total_time = end_time - start_time
train_hist['total_time'].append(total_time)

print("Avg per epoch time: %.2f, total %d epochs time: %.2f" % (torch.mean(torch.FloatTensor(train_hist['per_epoch_times'])), num_of_epochs, total_time))
print("Training finish!... save training results")

##TODO: save the generator and discriminator weights into .pkl files ##
torch.save(G.state_dict(), "results/generator.pkl")
torch.save(D.state_dict(), "results/discriminator.pkl")

with open('results/dcgan/train_hist.pkl', 'wb') as f:
    pickle.dump(train_hist, f)

##TODO: save training histograms ##
show_train_hist(train_hist, save=True, path='results/DCGAN_train_hist.png')

Started training...
Iteration 0 of 469
Iteration 1 of 469
Iteration 2 of 469
Iteration 3 of 469
Iteration 4 of 469
Iteration 5 of 469
Iteration 6 of 469
Iteration 7 of 469
Iteration 8 of 469
Iteration 9 of 469
Iteration 10 of 469
Iteration 11 of 469
Iteration 12 of 469
Iteration 13 of 469
Iteration 14 of 469
Iteration 15 of 469
Iteration 16 of 469
Iteration 17 of 469
Iteration 18 of 469
Iteration 19 of 469
Iteration 20 of 469
Iteration 21 of 469
Iteration 22 of 469
Iteration 23 of 469
Iteration 24 of 469
Iteration 25 of 469
Iteration 26 of 469
Iteration 27 of 469
Iteration 28 of 469
Iteration 29 of 469
Iteration 30 of 469
Iteration 31 of 469
Iteration 32 of 469
Iteration 33 of 469
Iteration 34 of 469
Iteration 35 of 469
Iteration 36 of 469
Iteration 37 of 469
Iteration 38 of 469
Iteration 39 of 469
Iteration 40 of 469
Iteration 41 of 469
Iteration 42 of 469
Iteration 43 of 469
Iteration 44 of 469
Iteration 45 of 469
Iteration 46 of 469
Iteration 47 of 469
Iteration 48 of 469
Iteration 

c:\program files\python35\lib\site-packages\ipykernel_launcher.py:3: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  This is separate from the ipykernel package so we can avoid doing imports until


Iteration 469 of 469
Iteration 470 of 469
Iteration 471 of 469
Iteration 472 of 469
Iteration 473 of 469
Iteration 474 of 469
Iteration 475 of 469
Iteration 476 of 469
Iteration 477 of 469
Iteration 478 of 469
Iteration 479 of 469
Iteration 480 of 469
Iteration 481 of 469
Iteration 482 of 469
Iteration 483 of 469
Iteration 484 of 469
Iteration 485 of 469
Iteration 486 of 469
Iteration 487 of 469
Iteration 488 of 469
Iteration 489 of 469
Iteration 490 of 469
Iteration 491 of 469
Iteration 492 of 469
Iteration 493 of 469
Iteration 494 of 469
Iteration 495 of 469
Iteration 496 of 469
Iteration 497 of 469
Iteration 498 of 469
Iteration 499 of 469
Iteration 500 of 469
Iteration 501 of 469
Iteration 502 of 469
Iteration 503 of 469
Iteration 504 of 469
Iteration 505 of 469
Iteration 506 of 469
Iteration 507 of 469
Iteration 508 of 469
Iteration 509 of 469
Iteration 510 of 469
Iteration 511 of 469
Iteration 512 of 469
Iteration 513 of 469
Iteration 514 of 469
Iteration 515 of 469
Iteration 516

Iteration 1235 of 469
Iteration 1236 of 469
Iteration 1237 of 469
Iteration 1238 of 469
Iteration 1239 of 469
Iteration 1240 of 469
Iteration 1241 of 469
Iteration 1242 of 469
Iteration 1243 of 469
Iteration 1244 of 469
Iteration 1245 of 469
Iteration 1246 of 469
Iteration 1247 of 469
Iteration 1248 of 469
Iteration 1249 of 469
Iteration 1250 of 469
Iteration 1251 of 469
Iteration 1252 of 469
Iteration 1253 of 469
Iteration 1254 of 469
Iteration 1255 of 469
Iteration 1256 of 469
Iteration 1257 of 469
Iteration 1258 of 469
Iteration 1259 of 469
Iteration 1260 of 469
Iteration 1261 of 469
Iteration 1262 of 469
Iteration 1263 of 469
Iteration 1264 of 469
Iteration 1265 of 469
Iteration 1266 of 469
Iteration 1267 of 469
Iteration 1268 of 469
Iteration 1269 of 469
Iteration 1270 of 469
Iteration 1271 of 469
Iteration 1272 of 469
Iteration 1273 of 469
Iteration 1274 of 469
Iteration 1275 of 469
Iteration 1276 of 469
Iteration 1277 of 469
Iteration 1278 of 469
Iteration 1279 of 469
Iteration 

Iteration 1605 of 469
Iteration 1606 of 469
Iteration 1607 of 469
Iteration 1608 of 469
Iteration 1609 of 469
Iteration 1610 of 469
Iteration 1611 of 469
Iteration 1612 of 469
Iteration 1613 of 469
Iteration 1614 of 469
Iteration 1615 of 469
Iteration 1616 of 469
Iteration 1617 of 469
Iteration 1618 of 469
Iteration 1619 of 469
Iteration 1620 of 469
Iteration 1621 of 469
Iteration 1622 of 469
Iteration 1623 of 469
Iteration 1624 of 469
Iteration 1625 of 469
Iteration 1626 of 469
Iteration 1627 of 469
Iteration 1628 of 469
Iteration 1629 of 469
Iteration 1630 of 469
Iteration 1631 of 469
Iteration 1632 of 469
Iteration 1633 of 469
Iteration 1634 of 469
Iteration 1635 of 469
Iteration 1636 of 469
Iteration 1637 of 469
Iteration 1638 of 469
Iteration 1639 of 469
Iteration 1640 of 469
Iteration 1641 of 469
Iteration 1642 of 469
Iteration 1643 of 469
Iteration 1644 of 469
Iteration 1645 of 469
Iteration 1646 of 469
Iteration 1647 of 469
Iteration 1648 of 469
Iteration 1649 of 469
Iteration 

...

Iteration 3827 of 469
Iteration 3828 of 469
Iteration 3829 of 469
Iteration 3830 of 469
Iteration 3831 of 469
Iteration 3832 of 469
Iteration 3833 of 469
Iteration 3834 of 469
Iteration 3835 of 469
Iteration 3836 of 469
Iteration 3837 of 469
Iteration 3838 of 469
Iteration 3839 of 469
Iteration 3840 of 469
Iteration 3841 of 469
Iteration 3842 of 469
Iteration 3843 of 469
Iteration 3844 of 469
Iteration 3845 of 469
Iteration 3846 of 469
Iteration 3847 of 469
Iteration 3848 of 469
Iteration 3849 of 469
Iteration 3850 of 469
Iteration 3851 of 469
Iteration 3852 of 469
Iteration 3853 of 469
Iteration 3854 of 469
Iteration 3855 of 469
Iteration 3856 of 469
Iteration 3857 of 469
Iteration 3858 of 469
Iteration 3859 of 469
Iteration 3860 of 469
Iteration 3861 of 469
Iteration 3862 of 469
Iteration 3863 of 469
Iteration 3864 of 469
Iteration 3865 of 469
Iteration 3866 of 469
Iteration 3867 of 469
Iteration 3868 of 469
Iteration 3869 of 469
Iteration 3870 of 469
Iteration 3871 of 469
Iteration 

Iteration 4569 of 469
Iteration 4570 of 469
Iteration 4571 of 469
Iteration 4572 of 469
Iteration 4573 of 469
Iteration 4574 of 469
Iteration 4575 of 469
Iteration 4576 of 469
Iteration 4577 of 469
Iteration 4578 of 469
Iteration 4579 of 469
Iteration 4580 of 469
Iteration 4581 of 469
Iteration 4582 of 469
Iteration 4583 of 469
Iteration 4584 of 469
Iteration 4585 of 469
Iteration 4586 of 469
Iteration 4587 of 469
Iteration 4588 of 469
Iteration 4589 of 469
Iteration 4590 of 469
Iteration 4591 of 469
Iteration 4592 of 469
Iteration 4593 of 469
Iteration 4594 of 469
Iteration 4595 of 469
Iteration 4596 of 469
Iteration 4597 of 469
Iteration 4598 of 469
Iteration 4599 of 469
Iteration 4600 of 469
Iteration 4601 of 469
Iteration 4602 of 469
Iteration 4603 of 469
Iteration 4604 of 469
Iteration 4605 of 469
Iteration 4606 of 469
Iteration 4607 of 469
Iteration 4608 of 469
Iteration 4609 of 469
Iteration 4610 of 469
Iteration 4611 of 469
Iteration 4612 of 469
Iteration 4613 of 469
Iteration 

Iteration 5307 of 469
Iteration 5308 of 469
Iteration 5309 of 469
Iteration 5310 of 469
Iteration 5311 of 469
Iteration 5312 of 469
Iteration 5313 of 469
Iteration 5314 of 469
Iteration 5315 of 469
Iteration 5316 of 469
Iteration 5317 of 469
Iteration 5318 of 469
Iteration 5319 of 469
Iteration 5320 of 469
Iteration 5321 of 469
Iteration 5322 of 469
Iteration 5323 of 469
Iteration 5324 of 469
Iteration 5325 of 469
Iteration 5326 of 469
Iteration 5327 of 469
Iteration 5328 of 469
Iteration 5329 of 469
Iteration 5330 of 469
Iteration 5331 of 469
Iteration 5332 of 469
Iteration 5333 of 469
Iteration 5334 of 469
Iteration 5335 of 469
Iteration 5336 of 469
Iteration 5337 of 469
Iteration 5338 of 469
Iteration 5339 of 469
Iteration 5340 of 469
Iteration 5341 of 469
Iteration 5342 of 469
Iteration 5343 of 469
Iteration 5344 of 469
Iteration 5345 of 469
Iteration 5346 of 469
Iteration 5347 of 469
Iteration 5348 of 469
Iteration 5349 of 469
Iteration 5350 of 469
Iteration 5351 of 469
Iteration 

Iteration 6048 of 469
Iteration 6049 of 469
Iteration 6050 of 469
Iteration 6051 of 469
Iteration 6052 of 469
Iteration 6053 of 469
Iteration 6054 of 469
Iteration 6055 of 469
Iteration 6056 of 469
Iteration 6057 of 469
Iteration 6058 of 469
Iteration 6059 of 469
Iteration 6060 of 469
Iteration 6061 of 469
Iteration 6062 of 469
Iteration 6063 of 469
Iteration 6064 of 469
Iteration 6065 of 469
Iteration 6066 of 469
Iteration 6067 of 469
Iteration 6068 of 469
Iteration 6069 of 469
Iteration 6070 of 469
Iteration 6071 of 469
Iteration 6072 of 469
Iteration 6073 of 469
Iteration 6074 of 469
Iteration 6075 of 469
Iteration 6076 of 469
Iteration 6077 of 469
Iteration 6078 of 469
Iteration 6079 of 469
Iteration 6080 of 469
Iteration 6081 of 469
Iteration 6082 of 469
Iteration 6083 of 469
Iteration 6084 of 469
Iteration 6085 of 469
Iteration 6086 of 469
Iteration 6087 of 469
Iteration 6088 of 469
Iteration 6089 of 469
Iteration 6090 of 469
Iteration 6091 of 469
Iteration 6092 of 469
Iteration 

Iteration 6786 of 469
Iteration 6787 of 469
Iteration 6788 of 469
Iteration 6789 of 469
Iteration 6790 of 469
Iteration 6791 of 469
Iteration 6792 of 469
Iteration 6793 of 469
Iteration 6794 of 469
Iteration 6795 of 469
Iteration 6796 of 469
Iteration 6797 of 469
Iteration 6798 of 469
Iteration 6799 of 469
Iteration 6800 of 469
Iteration 6801 of 469
Iteration 6802 of 469
Iteration 6803 of 469
Iteration 6804 of 469
Iteration 6805 of 469
Iteration 6806 of 469
Iteration 6807 of 469
Iteration 6808 of 469
Iteration 6809 of 469
Iteration 6810 of 469
Iteration 6811 of 469
Iteration 6812 of 469
Iteration 6813 of 469
Iteration 6814 of 469
Iteration 6815 of 469
Iteration 6816 of 469
Iteration 6817 of 469
Iteration 6818 of 469
Iteration 6819 of 469
Iteration 6820 of 469
Iteration 6821 of 469
Iteration 6822 of 469
Iteration 6823 of 469
Iteration 6824 of 469
Iteration 6825 of 469
Iteration 6826 of 469
Iteration 6827 of 469
Iteration 6828 of 469
Iteration 6829 of 469
Iteration 6830 of 469
Iteration 